# Toxicology Testing
For this assignment, you will use a chemical dataset to train a neural network to predict human reaction to exposure to certain compounds.

# Import Packages

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import deepchem as dc
from sklearn.metrics import accuracy_score
from tensorflow import keras
from keras import layers
import datetime

2024-10-05 19:14:41.712019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-05 19:14:42.484773: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-05 19:14:42.484872: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-05 19:14:42.484880: W 

# Set Environment

In [2]:
np.random.seed(456)
tf.random.set_seed(456)
# Only run this line if not yet loaded
%load_ext tensorboard

# Load and Prep Data

In [3]:
_, (train, valid, test), _ = dc.molnet.load_tox21()
train_X, train_y, train_w = train.X, train.y, train.w
valid_X, valid_y, valid_w = valid.X, valid.y, valid.w
test_X, test_y, test_w = test.X, test.y, test.w
train_y = train_y[:, 0]
valid_y = valid_y[:, 0]
test_y = test_y[:, 0]
train_w = train_w[:, 0]
valid_w = valid_w[:, 0]
test_w = test_w[:, 0]

[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerator
[19:14:59] DEPRECATION WARNING: please use MorganGenerat

# Significant Modification
Code originally written for Tensorflow v1, which has been deprecated for years. For this assignment, we will use Tensorflow v2. Re-writing the entire architecture, paying attention to the steps in the assignment, rather than the starter code. Will also be using Keras sequential architectures. One hint comes from [here](https://www.geeksforgeeks.org/hidden-layer-perceptron-in-tensorflow/).

# Define Model Placeholders

In [4]:
d = 1024
n_hidden = 50
learning_rate = .001
n_epochs = 10
batch_size = 100

# Create Model Architecture

In [5]:
model = keras.Sequential([ 
	layers.Dense(256, activation = 'relu', input_shape = [d]), 
	layers.BatchNormalization(), 
	layers.Dense(256, activation = 'relu'), 
	layers.Dropout(0.3), 
	layers.BatchNormalization(), 
	layers.Dense(1, activation = 'sigmoid') 
]) 

model.compile( 
	loss = 'binary_crossentropy', 
	optimizer = 'adam', 
	metrics = ['AUC'] 
) 

model.summary()

2024-10-05 19:15:05.079907: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-05 19:15:05.079931: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-10-05 19:15:05.079952: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-2c4d82d2-b129-4486-baa4-05c502f179ef): /proc/driver/nvidia/version does not exist
2024-10-05 19:15:05.080168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the a

# Tensorboard Setup

In [11]:
# https://www.tensorflow.org/tensorboard/get_started
# Clear any logs from previous runs
!rm -rf ./logs/

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir = log_dir)
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = model.fit(train_X, train_y, 
          epochs = n_epochs, 
          batch_size = batch_size, 
          verbose = 1, 
          validation_data = (valid_X, valid_y)) 
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Epoch 1/10
 6/63 [=>............................] - ETA: 0s - loss: 0.0141 - auc: 0.99942024-10-05 19:27:51.911966: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2024-10-05 19:27:51.911989: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
63/63 [==============================] - 1s 13ms/step - loss: 0.0196 - auc: 0.9920 - val_loss: 0.1576 - val_auc: 0.7641
Epoch 2/10
63/63 [==============================] - 1s 13ms/step - loss: 0.0196 - auc: 0.9892 - val_loss: 0.1766 - val_auc: 0.7948
Epoch 3/10
63/63 [==============================] - 1s 11ms/step - loss: 0.0155 - auc: 0.9928 - val_loss: 0.1885 - val_auc: 0.7340
Epoch 4/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0115 - auc: 0.9949 - val_loss: 0.1999 - val_auc: 0.7308
Epoch 5/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0135 - auc: 0.9969 - val_l

# Implement Mini-Batch Training

In [23]:
model.compile( 
	loss = 'cosine_similarity', 
	optimizer = 'adam', 
	metrics = ['AUC'] 
) 
model.fit(train_X, train_y, 
          epochs = n_epochs, 
          batch_size = batch_size, 
          verbose = 1, 
          validation_data = (valid_X, valid_y)) 

Epoch 1/10
63/63 [==============================] - 2s 15ms/step - loss: -0.0396 - auc: 0.9919 - val_loss: -0.0226 - val_auc: 0.7054
Epoch 2/10
63/63 [==============================] - 1s 12ms/step - loss: -0.0399 - auc: 0.9960 - val_loss: -0.0232 - val_auc: 0.7207
Epoch 3/10
63/63 [==============================] - 1s 11ms/step - loss: -0.0401 - auc: 0.9978 - val_loss: -0.0234 - val_auc: 0.7159
Epoch 4/10
63/63 [==============================] - 1s 12ms/step - loss: -0.0401 - auc: 0.9976 - val_loss: -0.0261 - val_auc: 0.7100
Epoch 5/10
63/63 [==============================] - 1s 11ms/step - loss: -0.0401 - auc: 0.9977 - val_loss: -0.0279 - val_auc: 0.7056
Epoch 6/10
63/63 [==============================] - 1s 12ms/step - loss: -0.0401 - auc: 0.9994 - val_loss: -0.0290 - val_auc: 0.7029
Epoch 7/10
63/63 [==============================] - 1s 11ms/step - loss: -0.0401 - auc: 0.9977 - val_loss: -0.0299 - val_auc: 0.7003
Epoch 8/10
63/63 [==============================] - 1s 11ms/step - lo

In [7]:
model.fit(train_X, train_y, 
          epochs = n_epochs, 
          batch_size = batch_size, 
          verbose = 1, 
          validation_data = (valid_X, valid_y)) 

Epoch 1/10
63/63 [==============================] - 2s 16ms/step - loss: 0.5618 - auc: 0.7213 - val_loss: 0.3263 - val_auc: 0.7112
Epoch 2/10
63/63 [==============================] - 1s 12ms/step - loss: 0.2459 - auc: 0.8754 - val_loss: 0.1732 - val_auc: 0.7101
Epoch 3/10
63/63 [==============================] - 1s 12ms/step - loss: 0.1214 - auc: 0.9439 - val_loss: 0.1401 - val_auc: 0.8167
Epoch 4/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0719 - auc: 0.9734 - val_loss: 0.1530 - val_auc: 0.7516
Epoch 5/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0502 - auc: 0.9772 - val_loss: 0.1460 - val_auc: 0.7707
Epoch 6/10
63/63 [==============================] - 1s 11ms/step - loss: 0.0427 - auc: 0.9858 - val_loss: 0.1656 - val_auc: 0.7261
Epoch 7/10
63/63 [==============================] - 1s 12ms/step - loss: 0.0316 - auc: 0.9844 - val_loss: 0.1779 - val_auc: 0.7235
Epoch 8/10
63/63 [==============================] - 1s 11ms/step - loss: 0.0369 - a

# TensorBoard Results

In [12]:
%tensorboard --logdir logs
# %tensorboard --log_dir logs/fit

Launching TensorBoard...

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2c4d82d2-b129-4486-baa4-05c502f179ef' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>